# **사전 준비**

In [ ]:
%%capture --no-stderr
%pip install -U langgraph
%pip install -U langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# 실습에 사용할 그래프의 상태 정의
# TypedDict = 타입 힌트

# Annotated는 런타임에 실제 동작하지 않고, 타입 검사기와 프레임워크에게 ‘설명/주석’ 역할을 하는 문법
# → “messages는 list 타입이다 그리고 add_messages라는 규칙이 붙어 있다”
# Python 입장에서는 단순 주석
# LangGraph 입장에서는 병합 방식 지시

class State(TypedDict):
  messages: Annotated[list, add_messages]

# 실습에서 사용할 그래프 인스턴스 생성
graph_builder = StateGraph(State)

# **챗봇 노드**

In [ ]:
from langchain_openai import ChatOpenAI

# 오픈 AI 클라이언트 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 오픈AI를 호출하여 응답을 받아온 뒤 상태값에 저장하여 반환하는 챗봇 함수 정의
def chatbot(state: State):
  return {"messages": [llm.invoke(state["messages"])]}

# 챗봇 노드 정의
graph_builder.add_node("chatbot", chatbot)

In [ ]:
from langgraph.graph import StateGraph, START, END

# 진입 지점
graph_builder.add_edge(START, "chatbot")

# 종료 지점
graph_builder.add_edge("chatbot", END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
# 무한 루프
while True:
  # 사용자의 질의 입력받기
  # input() 함수는 사용자 입력을 기다림
  # "User: " 라는 안내 문구를 보여주고 입력받은 값을 user_input 변수에 저장
  user_input = input("User: ")

  # 사용자가 quit 또는 exit, q를 입력한다면 루프 종료
  if user_input.lower() in ["quit", "exit", "q"]:
    print("Goodbye!")
    break

  # 사용자의 입력을 그래프에 전달하여 정의된 흐름 실행
  # graph.stream() → LangGraph 그래프 실행
  # ("user", user_input) → 튜플로 “사용자가 입력했다” 표시
  # 결괏값 event에 저장
  # {"messages": ("user", user_input)} 정의한 State
  for event in graph.stream({"messages": ("user", user_input)}):
    for value in event.values():
      print("Assistant:", value["messages"][-1].content)

# **그래프 시각화**

In [ ]:
from IPython.display import Image, dsiplay

display(Image(graph.get_graph().draw_mermaid_png()))

# **Tavily 검색 엔진 세팅**

In [ ]:
%%capture --no-stderr
%pip install -U tavily-python
%pip install -U langchain_community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily 검색 엔진을 도구로 정의
tool = TavilySearchResults(max_results=2)
tools = [tool]

# 호출 예시
tool.invoke("내일 대한민국 서울의 날씨는?")

# **외부 검색 도구 노드**

In [ ]:
from typing import Annotated
from langchain_openai import ChatOpneAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

# 그래프 상태 정의
class State(TypedDict):
  messages: Annotated[list, add_messages]

# 그래프 정의
graph_builder = StateGraph(State)

# 오픈AI 클라이언트 정의
llm = ChatOpenAI(model="gpt-4o-mini")
# 오픈AI 클라이언트에 Tavily 검색 엔진 도구 할당
llm_with_tools = llm.bind_tools(tools)

# 챗봇 함수 정의
def chatbot(state: State):
  return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 그래프에 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)